 # MTH8408 : Méthodes d'optimisation et contrôle optimal
 ## Laboratoire 1: Outils pour l'algèbre linéaire et l'optimisation
Tangi Migot

In [1]:
using Pkg; 
Pkg.activate("jump")

  Activating project at `c:\Users\tangi\Documents\cvs\MTH8408\lab1\jump`


## JuMP: un langage de modélisation

### JuMP ?
JuMP est le language de modélisation pour l'optimisation natif dans Julia. En d'autres termes, on peut modéliser des problèmes d'optimisation en Julia grâce à ce package.

JuMP fait partie de l'organisation [jump-dev](https://jump.dev/) qui propose un ensemble d'outils concernant l'optimisation.

On pourra trouver beaucoup d'informations et d'exemples dans la documentation de JuMP [jump.dev/JuMP.jl/stable/](https://jump.dev/JuMP.jl/stable/)

### Installer JuMP
Comme on l'a fait pour `LinearAlgebra` il faut installer le package `JuMP` via le *package manager*.

In [ ]:
] add JuMP

Ensuite, on peut ajouter `JuMP` a notre environement et commencer à l'utiliser.

In [2]:
using JuMP

### Installer un solveur
Dans cet exemple, on va ajouter [`Ipopt`](https://en.wikipedia.org/wiki/IPOPT) qui est un package qui fournit un solveur pour l'optimisation nonlinéaire.

In [ ]:
] add Ipopt
#ou alors:
#using Pkg
#Pkg.add("Ipopt")

In [3]:
using Ipopt

In [4]:
? ipopt

ErrorException: syntax: invalid identifier name "?"

On installe aussi `MathOptInterface` qui permet de communiquer entre le modèle et le solveur.

In [5]:
using MathOptInterface

### Structure d'un modèle JuMP
Tous les modèles écrits avec JuMP ont la même structure:
- un objet `Model`, c'est l'initialisation de notre modèle.
- un objet `Optimizer`, qui spécifie un solveur.
- `Variables`, les variables du problème
- `Objective`, la fonction objectif
- `Constraints`, les contraintes.

#### Le modèle
Tous les problèmes modéliser avec JuMP sont des objets `Model`. Cet objet sera ensuite toujours mentionné lorsque l'on ajoutera des variables, la fonction objectif et des contraintes.

In [6]:
MonModel = Model()

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

Il est également possible d'associer un solveur à ce modèle.

In [7]:
MonModel = Model(with_optimizer(Ipopt.Optimizer)) #utilise Ipopt comme solveur

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt

#### Les variables
Une première donnée importante de notre problème d'optimisation est la variable.

In [17]:
@variable(MonModel, 0 <= x <= 10)
#unregister(MonModel, :x)

A ce stade, on peut aussi spécifier des contraintes de bornes sur les variables.

In [18]:
n = 10
l = [1; 2; 3; 4; 5; 6; 7; 8; 9; 10]
u = [10; 11; 12; 13; 14; 15; 16; 17; 18; 19]
@variable(MonModel, l[i] <= x[i=1:n] <= u[i])

10-element Vector{VariableRef}:
 x[1]
 x[2]
 x[3]
 x[4]
 x[5]
 x[6]
 x[7]
 x[8]
 x[9]
 x[10]

ou plus simplement

In [19]:
@variable(MonModel, xx[i=1:n] >= 0)

ErrorException: An object of name xx is already attached to this model. If this
    is intended, consider using the anonymous construction syntax, e.g.,
    `x = @variable(model, [1:N], ...)` where the name of the object does
    not appear inside the macro.

    Alternatively, use `unregister(model, :xx)` to first unregister
    the existing name from the model. Note that this will not delete the
    object; it will just remove the reference at `model[:xx]`.


#### La fonction objectif
La fonction objectif se définit à l'aide de `@objective` et a trois arguments: le nom du modèle, le type d'optimisation (min ou max), et la fonction.

In [20]:
c = [2; 3; 4; 5] 
@objective(MonModel, Min, sum(c[i]*x[i] for i in 1:4))

2 x[1] + 3 x[2] + 4 x[3] + 5 x[4]

On peut aussi l'écrire sous forme de problème de maximisation.

In [22]:
@objective(MonModel, Max, sum(c[i]*x[i] for i in 1:4))

2 x[1] + 3 x[2] + 4 x[3] + 5 x[4]

Si la fonction objectif est nonlinéaire, on peut être plus précis et utiliser `@NLobjective`.

In [24]:
@NLobjective(MonModel, Min, exp(x[1]))

#### Les contraintes

In [25]:
TonModel = Model()
@variable(TonModel, x) #on peut avoir plusieurs types de variables
@variable(TonModel, y)
@constraint(TonModel, 5*x + 3*y <= 5) #une première contrainte

5 x + 3 y <= 5.0

Il peut être utile de nommer une contrainte pour s'y réferer ensuite.

In [26]:
@constraint(TonModel, un_nom_de_contrainte, 6*x + 4*y >= 5)

un_nom_de_contrainte : 6 x + 4 y >= 5.0

On peut aussi ajouter plusieurs contraints simultanément avec @constraints

In [27]:
@constraints(model, begin
           2x <=  1
            x >= -1
       end)

UndefVarError: UndefVarError: model not defined

In [28]:
TonModel[:un_nom_de_contrainte]

un_nom_de_contrainte : 6 x + 4 y >= 5.0

Si la contrainte est nonlinéaire, on peut être plus précis et utiliser `@NLconstraint`.

In [29]:
@NLconstraint(MonModel, x^2 == 0)

ErrorException: Variable in nonlinear expression does not belong to the corresponding model

On peut également introduire des contraintes plus génériques à l'aide de fonctions.

In [30]:
a = [1; -3; 5; 7] 
@variable(TonModel, w[1:4])
@constraint(TonModel, sum(a[i]*w[i] for i in 1:4) <= 3)

w[1] - 3 w[2] + 5 w[3] + 7 w[4] <= 3.0

On peut utiliser une boucle pour définir plusieurs contraintes.

In [31]:
@constraint(TonModel, conRef3[i in 1:3], 6*x + 4*y >= 5*i)
#= ou identiquement:
for i in 1:3
    @constraint(yourModel, 6*x + 4*y >= 5*i)
end
=#

3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 conRef3[1] : 6 x + 4 y >= 5.0
 conRef3[2] : 6 x + 4 y >= 10.0
 conRef3[3] : 6 x + 4 y >= 15.0

#### Exemple d'un code complet: Optimisation linéaire
On résout ici un problème d'optimisation linéaire
$$
\min_{x \in R^n} - x - 2y \ \text{sujet à} \ x + y \leq 1, 0 \leq x,y \leq 1.
$$

In [32]:
model = Model(with_optimizer(Ipopt.Optimizer))
# Old syntax: model = Model(solver=GLPKSolverLP(msg_lev = 4)))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt

On définit les variables.

In [33]:
@variable(model, 0 <= x <= 1)
@variable(model, 0 <= y <= 1)

y

La contrainte $x + y \leq 1$.

In [34]:
@constraint(model, x + y <= 1)

x + y <= 1.0

Enfin, on ajoute la fonction objectif.

In [35]:
@objective(model, Min, -x - 2y)

-x - 2 y

Pour résoudre ce problème, on peut utiliser la fonction `optimize`.

In [36]:
JuMP.optimize!(model) # Old syntax: status = JuMP.solve(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        2
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality co

Une fois la résolution terminée, on peut vérifier le status de la résolution.

In [37]:
@show JuMP.has_values(model)
@show JuMP.termination_status(model) == MathOptInterface.LOCALLY_SOLVED
@show JuMP.primal_status(model) == MathOptInterface.FEASIBLE_POINT
@show JuMP.dual_status(model) == MathOptInterface.FEASIBLE_POINT

JuMP.has_values(model) = true
JuMP.termination_status(model) == MathOptInterface.LOCALLY_SOLVED = true
JuMP.primal_status(model) == MathOptInterface.FEASIBLE_POINT = true
JuMP.dual_status(model) == MathOptInterface.FEASIBLE_POINT = true


true

On peut également faire une analyse sur la solution obtenue.

In [38]:
@show JuMP.value(x)              # Old syntax: getvalue(x)
@show JuMP.value(y)              # Old syntax: getvalue(y)
@show JuMP.objective_value(model)       # Old syntax: getobjectivevalue(model)

JuMP.value(x) = -4.860843992474532e-10
JuMP.value(y) = 1.0000000083700527
JuMP.objective_value(model) = -2.000000016254021


-2.000000016254021

#### Exemple d'un code complet: Optimisation nonlinéaire
$$
\min_{x,y} exp(x)+y \text{ s.à } exp(x) + sin(x) <= 0
$$

In [39]:
model = Model(with_optimizer(Ipopt.Optimizer))
@variable(model, x)
@variable(model, y)

@NLobjective(model, Min, exp(x)+y)
@NLconstraint(model, exp(x)+sin(x) <=0)

(exp(x) + sin(x)) - 0.0 <= 0

In [40]:
optimize!(model)
println("x = ", value(x), " y = ", value(y))

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        1
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        1

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.0000000e+00 1.00e+00 1.20e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

In [41]:
termination_status(model)

NORM_LIMIT::TerminationStatusCode = 17

In [43]:
vals = zeros(2)
x_index = JuMP.index(x)
y_index = JuMP.index(y)

MathOptInterface.VariableIndex(2)

In [44]:
d = NLPEvaluator(model)

An NLPEvaluator with available features:
  * :Grad
  * :Jac
  * :JacVec
  * :ExprGraph
  * :Hess
  * :HessVec

In [45]:
MathOptInterface.initialize(d, [:Grad])

In [46]:
MathOptInterface.eval_objective(d, zeros(2))

1.0

In [47]:
∇f = zeros(2)
MathOptInterface.eval_objective_gradient(d, ∇f, zeros(2))
∇f

2-element Vector{Float64}:
 1.0
 1.0

In [48]:
J = zeros(1,2)
MathOptInterface.eval_constraint_jacobian(d, J, zeros(2))
J

1×2 Matrix{Float64}:
 2.0  0.0

#### Plus d'information
Dans ce cours, nous n'allons pas uniquement nous concentrer sur des problèmes linéaires, mais aussi nonlinéaire.
Vous trouverez plus d'informations pour ce cas dans la documentation de jump-dev:

[https://jump.dev/JuMP.jl/stable/nlp/#Nonlinear-Modeling-1](https://jump.dev/JuMP.jl/stable/nlp/#Nonlinear-Modeling-1)

et la documentation de MathOptInterface:

[https://jump.dev/MathOptInterface.jl/stable/](https://jump.dev/MathOptInterface.jl/stable/)

Et ça sera le sujet du projet !